## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/TextMining_WordEmbedding

/content/drive/MyDrive/TextMining_WordEmbedding


In [ ]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http:

In [ ]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [ ]:
corpus[:50]

['애니메이션에서나 볼 수 있었던 자동차 변신 로봇이 일본에서 실제로 만들어진다',
 "11일 소프트뱅크 계열 로봇 제조업체인 아스라테크(Asratec)와 산세이 테크놀로지, 브레이브 로보틱스가 3사 합작으로 변신 로봇 '제이 다이트 라이드(J-deite RIDE)'를 제작한다고 발표했다",
 '이 로봇은 아스라테크와 브레이브 로보틱스가 참여한 로봇 제조 계획 "프로젝트 제이 다이트\'의 일환으로 지난해 6월부터 개발이 진행되고 있다',
 "하드웨어 설계 ·제작은 브레이브 로보틱스가 맡고 제어용 소프트웨어 '브이시도(V-Sido)'의 개발은 아스라테크가 담당한다",
 '산세이 테크놀로지는 로봇의 양산화를 위해서 합류했다',
 "'제이 다이트 라이드'는 내년에 완성돼 테마 파크와 각종 이벤트 참여, 기업 홍보 등의 목적으로 쓰이게 된다",
 "이후에는 기체의 크기를 키워 2020년에는 5m에 달하는 '제이 다이트 오리지널'로 재탄생할 예정이다",
 "디자인은 애니메이션 '기동전사 건담', '달려라 번개호' 등 유명 애니메이션에 등장하는 로봇을 창조한 오오카와라 쿠니오가 맡았다",
 "오오카와라는 '제이 다이트 라이드' 외에 프로젝트에서 제작할 예정인 다른 로봇의 디자인도 맡는다",
 "'제이다이트 라이드'의 머리 부분 스케치",
 '언뜻 보기에도 건담이 연상된다',
 '이 로봇은 10초만에 자동차로 변신할 수 있으며 전체 길이는 약 4m이다',
 '로봇 모드에서는 두 다리로 걸을 수 있다',
 '차량 모드에서는 평범한 자동차와 마찬가지로 사람이 운전석에 앉아 조작하면 된다',
 "'철인28호'처럼 로봇과 떨어져 원격 조종을 해도 된다",
 '최대 2명이 탈 수 있다',
 '로봇이 두 다리로 걷는 속도는 시속 5km에 불과하지만 발 부분의 바퀴를 이용하면 한시간에 최고 30km를 갈 수 있다',
 '차로 변신했을 때 최고 속도는 시속 60km이다',
 "한편 '제이다이트 라이드'의 1.5m짜리 시제품은 14일(현지시간) 미국 올랜도에서 열리는 '국제 놀이기구 엑스

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]
#Number가 기사 제목에서 유의미하므로 stop word에 포함하지 않았고, Alpha를 없애고 싶었지만 1.5m의 m과 같은 의미 있는 것도 있어 제거하지 않았다

In [ ]:
okt = Okt()

In [ ]:
okt.pos('이 로봇은 10초만에 자동차로 변신할 수 있으며 전체 길이는 약 4m이다')

[('이', 'Noun'),
 ('로봇', 'Noun'),
 ('은', 'Josa'),
 ('10초', 'Number'),
 ('만에', 'Foreign'),
 ('자동차', 'Noun'),
 ('로', 'Josa'),
 ('변신', 'Noun'),
 ('할', 'Verb'),
 ('수', 'Noun'),
 ('있으며', 'Adjective'),
 ('전체', 'Noun'),
 ('길이', 'Noun'),
 ('는', 'Josa'),
 ('약', 'Noun'),
 ('4', 'Number'),
 ('m', 'Alpha'),
 ('이다', 'Josa')]

In [ ]:
okt.pos('데이터 사이언스 랩은 텍스트 마이닝에 대해서도 공부할 수 있는 좋은 학회입니다.')

[('데이터', 'Noun'),
 ('사이언스', 'Noun'),
 ('랩', 'Noun'),
 ('은', 'Josa'),
 ('텍스트', 'Noun'),
 ('마', 'Noun'),
 ('이닝', 'Noun'),
 ('에', 'Josa'),
 ('대해', 'Noun'),
 ('서도', 'Noun'),
 ('공부', 'Noun'),
 ('할', 'Verb'),
 ('수', 'Noun'),
 ('있는', 'Adjective'),
 ('좋은', 'Adjective'),
 ('학회', 'Noun'),
 ('입니다', 'Adjective'),
 ('.', 'Punctuation')]

In [ ]:
okt.morphs('이 로봇은 10초만에 자동차로 변신할 수 있으며 전체 길이는 약 4m이다')

['이',
 '로봇',
 '은',
 '10초',
 '만에',
 '자동차',
 '로',
 '변신',
 '할',
 '수',
 '있으며',
 '전체',
 '길이',
 '는',
 '약',
 '4',
 'm',
 '이다']

In [ ]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.


def CustomTokenization(sentence):
    token = []
    stop = ['Josa','Suffix','Punctuation', 'Foreign'] 
    for i, j in okt.pos(sentence, stem = True, norm = True):
        if j in stop:
          continue
        token.append(i)   
    return token

#sentence를 넣으면 토큰화된 리스트가 반환되게 의도

In [ ]:
CustomTokenization('이 로봇은 10초만에 자동차로 변신할 수 있으며 전체 길이는 약 4m이다')

['이', '로봇', '10초', '자동차', '변신', '하다', '수', '있다', '전체', '길이', '약', '4', 'm']

In [ ]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [ ]:
corpus_tokenized

[['애니메이션', '볼', '수', '있다', '자동차', '변신', '로봇', '일본', '실제', '만들어지다'],
 ['11일',
  '소프트뱅크',
  '계열',
  '로봇',
  '제조업체',
  '아스',
  '테크',
  'Asratec',
  '오다',
  '산세이',
  '테크놀로지',
  '브레이브',
  '로보틱스',
  '3',
  '사',
  '합작',
  '변신',
  '로봇',
  '제이',
  '다이트',
  '라이드',
  'J',
  'deite',
  'RIDE',
  '를',
  '제작',
  '다',
  '고',
  '발표',
  '하다'],
 ['이',
  '로봇',
  '아스',
  '테크',
  '브레이브',
  '로보틱스',
  '참여',
  '로봇',
  '제조',
  '계획',
  '프로젝트',
  '제이',
  '다이트',
  '의',
  '일환',
  '지난해',
  '6월',
  '개발',
  '진행',
  '되다',
  '있다'],
 ['하드웨어',
  '설계',
  '제작',
  '브레이브',
  '로보틱스',
  '맡다',
  '제어',
  '용',
  '소프트웨어',
  '브이',
  '시도',
  'V',
  'Sido',
  '의',
  '개발',
  '아스',
  '테크',
  '담당',
  '다'],
 ['산세이', '테크놀로지', '로봇', '양산', '위해', '합류', '하다'],
 ['제이',
  '다이트',
  '라이드',
  '늘다',
  '내년',
  '완성',
  '돼다',
  '테마',
  '파크',
  '각종',
  '이벤트',
  '참여',
  '기업',
  '홍보',
  '등',
  '목적',
  '쓰이다',
  '되다'],
 ['이후',
  '기체',
  '크기',
  '키우다',
  '2020년',
  '5',
  'm',
  '달',
  '하다',
  '제이',
  '다이트',
  '오',
  '리지',
  '널',
  '로',
  '재탄생',
  '하다',
  '

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]
#유의미하지만 한 번만 나온 숫자, 단위와 같은 것들도 살리려면 min_count가 0이어야 할 것 같다

In [ ]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(min_count=0) #유의미하지만 한 번만 나온 숫자, 단위와 같은 것들도 살리려면 min_count가 0이어야 할 것 같다

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 4062


In [ ]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 50 )

(778155, 869650)

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용
#50번 정도 학습하니까 스마트폰 주변 단어로 '모바일', '특허', '출원', '특허청' 등 관련 단어가 나왔다
#바이러스 주변 단어로도 '지카' '감염', '쥐' 등등

model_no_pretrained.wv.most_similar('스마트폰')

[('출원', 0.9938747882843018),
 ('특허', 0.9845036268234253),
 ('접다', 0.9799385070800781),
 ('풀다', 0.9794381856918335),
 ('관심', 0.9792090058326721),
 ('모바일', 0.978391170501709),
 ('현실', 0.9779974222183228),
 ('열쇠', 0.9768111705780029),
 ('기회', 0.9758962392807007),
 ('란', 0.9756847620010376)]

In [ ]:
model_no_pretrained.wv.most_similar('바이러스')

[('카', 0.988967776298523),
 ('지', 0.9886401295661926),
 ('쥐다', 0.9764567017555237),
 ('감염', 0.9714195728302002),
 ('쥐', 0.9549745321273804),
 ('고환', 0.9519431591033936),
 ('수컷', 0.9449929594993591),
 ('생식기관', 0.9426283836364746),
 ('정모', 0.9420958757400513),
 ('정액', 0.9158856272697449)]

In [ ]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size=200, min_count=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 4062


In [ ]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31690


In [ ]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 50)

(860618, 869650)

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용


model_pretrained.wv.most_similar('스마트폰')

[('접다', 0.9429429173469543),
 ('코드명', 0.8472867012023926),
 ('웨건', 0.827989935874939),
 ('hybrid', 0.8251611590385437),
 ('multicore', 0.8241852521896362),
 ('토요타', 0.819892168045044),
 ('얹다', 0.8196855783462524),
 ('마츠다', 0.8192039728164673),
 ('5일', 0.8168772459030151),
 ('카터', 0.813240647315979)]

In [ ]:
model_pretrained.wv.most_similar('바이러스')

[('박테리아', 0.7548943758010864),
 ('병원체', 0.7467655539512634),
 ('생식기관', 0.7133197784423828),
 ('감염', 0.703366756439209),
 ('종양', 0.6659607887268066),
 ('백신', 0.6606658697128296),
 ('항생제', 0.6393835544586182),
 ('항체', 0.6387474536895752),
 ('생식세포', 0.6370121240615845),
 ('기생충', 0.6357135772705078)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]
## 스마트폰은 잘 모르겠으나 바이러스 주변 단어를 확인해보니 사전 학습이 된 모델이 훨씬 좋은 성능을 보였다
## 단어가 무엇이냐에 따라 사전 학습된 모델의 성능이 다르게 나타나는 듯 하다
## 그 이유로 미루어보자면 '바이러스'와 같은 단어는 같이 연관지어서 나올 수 있는 단어가 의학 용어로 살짝 굳어져 있지만 '스마트폰'과 같은 단어는 계속해서 새로운 신제품이 나오고 있어 주변에 올 수 있는 단어가 계속 새롭게 추가되는 것과 연관이 있을 것 같다

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

#PCA로 했을 때는 잘 나누어졌는지는 잘 모르겠지만 단어들이 모여있는 곳을 확대해보니 그렇게 연관성이 깊어보이지는 않았다. 하지만 바깥 부분에 입자, 태양, 표면, 세포과 유사하게 임베딩되었다. 재밌는 건 아우디와 소바가 임베딩되었다는 점? 왜지?

#t-SNE 단어들이 더 가운데로 모임. '체포'와 '오후'가 같이 붙어있네? '방정식'-'유용하다', '광화문'-'남미'

#perplexity 5로 설정하니까 조금 더 잘 나누어지는 느낌이다! 특히 러시아,네덜란드,노르웨이,프랑스 등 유럽 국가들이 한 그룹으로 군집된 것이 눈에 띈다
#김성근-나노입자?-알츠하이머병이 연결되어 있었다...

In [ ]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)
list(set(sum(corpus_tokenized,[])))

['네이처',
 '네이쳐',
 '참전',
 '아산병원',
 '남성',
 '결함',
 '후송',
 '863',
 '무늘',
 '재현',
 'TGCCGAT',
 '띠기',
 '2004년',
 '마약',
 '떠올리다',
 '마이크로미터',
 '투명',
 '주최',
 '세기',
 '펠로우',
 '날카롭다',
 '방법',
 'm',
 '독일',
 '근지점',
 '적합하다',
 '정종',
 '과실',
 '를',
 '법',
 '심태섭',
 '이로',
 '13일',
 '화대',
 '2016년',
 '수소',
 '23',
 '적도',
 '땅',
 '강도',
 '음향학',
 '녹음',
 '127만',
 '합삭',
 '이란',
 '토론',
 '같다',
 '읽다',
 '면적',
 '이름',
 '마찰',
 '7,000회',
 '뜻밖',
 '이처럼',
 '개',
 '중반',
 'IO',
 '착안',
 '글리',
 '통해',
 '일정하다',
 '질량',
 '비판',
 '임',
 '드리우다',
 '같이',
 'Biologically',
 '틀다',
 '서울대',
 '계열',
 '충분하다',
 '필요시',
 '코타',
 '쿠니',
 '듀크',
 '휴대',
 '가계도',
 '트레일러',
 'Shooting',
 '세라',
 '뇌출혈',
 '병',
 '로켓',
 '전투',
 '강조',
 '보기',
 '행성',
 '기여',
 '내놓다',
 '여러',
 '농작물',
 '본드',
 '값',
 '익',
 '래커',
 '튕겨내',
 '아이폰',
 '꾸리다',
 '재교육',
 '우수하다',
 '면서',
 '얼음',
 'D',
 '망',
 '시도',
 '줄어들다',
 '물론',
 '대기',
 '로슈',
 '잊혀지다',
 '여전하다',
 '사인',
 '게임기',
 '흰색',
 'Y',
 '행사',
 'SF',
 '언뜻',
 'Expo',
 '응용',
 '일반인',
 '팬텀',
 '표적',
 '혁신',
 '유도',
 '망토',
 'Neuman',
 '생쥐',
 '특수부대',
 '외과',
 '바깥쪽',
 '논문

In [ ]:
vector_pretrained = model_pretrained
vector_no_pretrained = model_no_pretrained

In [ ]:
# embedding 결과 가져오기

word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(vector_pretrained.wv[i])
# model_no_pretrained.wv[i] or 로 하면 왜 안 되지??


In [ ]:
pca = PCA(random_state=0).fit_transform(vector)[:,:2]
print(pca.shape)
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:,:2]
print(tsne.shape)

(4062, 2)
(4062, 2)


In [ ]:
import plotly
import plotly.graph_objs as go
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
X, t = make_swiss_roll(n_samples = 4968, noise = 0)

In [ ]:
print(X.shape)
print(t.shape)

(4968, 3)
(4968,)


In [ ]:
print(t[:10])

[14.06734672  5.73653929 11.21477077  6.97610011 11.3808673  14.01658009
  4.75547647 11.14481179 12.38286776  9.65379662]


In [ ]:
color = (t - t.min()) / (t.max() - t.min())

In [ ]:
# 시각화해보세요!
# PCA
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = unique_text, #['point #{}'.format(i) for i in range(pca.shape[0])],
    mode='markers+text', 
    
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    ),
   
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning: plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin

  DeprecationWarning,


In [ ]:
# t-SNE
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = unique_text, #['point #{}'.format(i) for i in range(tsne.shape[0])],
    mode='markers+text',
    marker=dict(
        size=5,
        
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

#perplexity 50, lr도 설정해서 해봤는데 살짝 애매

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [ ]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 5, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = unique_text, #['point #{}'.format(i) for i in range(tsne.shape[0])],
    mode='markers+text',
    marker=dict(
        size=5,
        
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()



/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!